## <center> Web-scrapping of higher education institution data

This program is written to web-scrape data of higher education institutions from Niche, a website that publishes higher education rankings. The data webscrapped is as of March 23, 2022. These data will be used to explore features that make up this ranking and that may have a relationship with the median earnings post graduation feature considered in this analysis. Institutions that rank more highly in this ranking, on average, are expected to have higher median earnings post graduation under the assumption that a higher ranking implies higher educational quality.

Below is an image of a Niche web page and the publicly available data that is web-scrapped using this program.

Features web-scrapped include:
* Institution ranking
* Institution name
* Number of reviews
* User review score (from 1-5)
* Featured student review
* Overall Niche Grade
* Student acceptance rate
* Net price
* SAT range

![image](./assets/mit.png)

### Import libraries

In [1]:
import schedule
from datetime import time
from time import sleep
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np
import os
import sqlite3

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

### Scrape the required data from the Niche Best Colleges page

Perform the following operations:
* Set up lists as containers that will store the scrapped data
* Set up a Chrome webdriver to access the page that will scrapped
* Iterate through 33 URLs found on the Niche's [Best Colleges](https://www.niche.com/colleges/search/best-colleges/) page.
* Scrapped data only of institutions that have `search-result-badge`, `search-result-feature__author`, `search-result-feature__teaser`, and `search-result-fact-list` attributes, or the data intended to be web-scrapped
* Use list comprehensions to clean the data within each loop iteration

In [2]:
ranks = []
stu_reviews = []
stu_review_cnt = []
stu_ratings = []
niche_grades = []
inst_facts = []

In [3]:
for i in range(1,34):
    ser = Service('chromedriver/chromedriver')
    op = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=ser, options=op)

    driver.maximize_window()
    URL = f'https://www.niche.com/colleges/search/best-colleges/?page={i}'
    driver.get(URL)

    html = driver.page_source
    soup = BeautifulSoup(html)

    try:
        # Get data of ranked institutions
        cards = [x for x in soup.findAll('div', {'class':'card__inner'}) if 'search-result-badge' in str(x) and
                'search-result-feature__author' in str(x) and 'search-result-feature__teaser' in str(x) and 
                'search-result-fact-list' in str(x)]

        # Get names of ranked institutions
        names = [str(name).split('"search-result__title">')[1].split('<')[0] for name in cards]

        # Get rankings of ranked institutions
        rankings = [str(ranking).split('search-result-badge">')[1].split('<')[0] for ranking in cards]

        [ranks.append((name,rank)) for name,rank in zip(names,rankings)]

        # Get featured student reviews of ranked institutions
        reviews = [str(review).split('"search-result-feature__author">')[1].split('</strong>\xa0<span>')[1].\
                   replace('</span>','').replace('<span>','').replace('<span class="a11y-hide-when-needed">','').\
                   replace("\\","").split('<span class=')[0].replace("\\",'') for review in cards]
        
        [stu_reviews.append((name,review)) for name,review in zip(names,reviews)]

        # Get number of reviews
        review_cnt = [str(count).split('<!-- --> <!-- -->Reviews</span>')[0].split('Read <!-- -->')[1] 
                      for count in cards]

        [stu_review_cnt.append((name,review_num)) for name,review_num in zip(names,review_cnt)]
        
        # Get the ratings per institution
        ratings = [str(rating).split('Rating <!-- -->')[1].split('<!-- --> out of')[0] for rating in cards]
        
        [stu_ratings.append((name,rating)) for name,rating in zip(names,ratings)]

        # Get the grade-level per institution assigned by Niche
        grades = [str(grade).split('Overall Niche Grade')[2].split('<!-- -->: <!-- -->')[1].split('<!-- -->')[0] 
                  for grade in cards]

        [niche_grades.append((name,grade)) for name,grade in zip(names,grades)]
        
        # Get the facts data and labels
        facts = [str(str(fact).split('Overall Niche Grade')[2].split('</p><p>')[1:]).replace('<!-- -->','').\
                 replace('</p></div></div>','').replace(',','').replace('[','').replace(']','').replace("' '",', ')
                 for fact in cards]
        
        [inst_facts.append((name,fact)) for name,fact in zip(names,facts)]
        
        if len(cards) == 0:
            print("Page number not scrapped:", i)
        
    except:
        ranks.append('No value')
        stu_reviews.append('No value')
        stu_review_cnt.append('No value')
        stu_ratings.append('No value')
        niche_grades.append('No value')
        inst_facts.append('No value')

    driver.close()
    sleep(150)

Verify that all list containers are of the same length.

In [4]:
assert len(ranks) == len(stu_reviews) == len(stu_review_cnt) == len(stu_ratings) == len(niche_grades) \
    == len(inst_facts)

### Save webscrapped data

In the following function, create a data frame of the institution name and the web-scrapped data. Use this function to merge all feature-specific data frames into one data frame.

In [5]:
def create_df(lister,name):
    df = pd.DataFrame([i[0] for i in lister],[i[1] for i in lister], columns = ['instnm'])
    df.reset_index(inplace = True)
    df.rename(columns={'index':name}, inplace = True)
    
    return df

In [6]:
df = create_df(ranks,'ranks').merge(create_df(stu_reviews,'stu_reviews'), how = 'outer', on = 'instnm').\
    merge(create_df(stu_review_cnt,'stu_review_cnt'), how = 'outer', on = 'instnm').\
    merge(create_df(stu_ratings,'stu_ratings'), how = 'outer', on = 'instnm').\
    merge(create_df(niche_grades,'niche_grades'), how = 'outer', on = 'instnm').\
    merge(create_df(inst_facts,'inst_facts'), how = 'outer', on = 'instnm')

In [7]:
df.drop_duplicates(subset = ['instnm'], inplace = True)

Compute the polarity and subjectivity of student reviews to identify potential reasons for positive and negative reviews and whether they relate to graduate wages post graduation.

In [8]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [9]:
df['review_polarity'] = [nlp(review)._.polarity for review in df.stu_reviews]

In [10]:
df['review_subjectivity'] = [nlp(review)._.subjectivity for review in df.stu_reviews]

### Create a SQLite database to store the data

In [11]:
sqlite_db = './data/webscrapping_profile_feats.sqlite'
conn = sqlite3.connect(sqlite_db)

In [12]:
df.to_sql('niche_webscrapping',
            con=conn,
            if_exists='replace',
            index=False)

### Display the first five rows of the data frame

In [13]:
sql = """
SELECT *
FROM niche_webscrapping
LIMIT 5
"""

pd.read_sql(sql, con = conn)

,ranks,instnm,stu_reviews,stu_review_cnt,stu_ratings,niche_grades,inst_facts,review_polarity,review_subjectivity
0,#1 Best Colleges in America,Massachusetts Institute of Technology,Currently reaching near the middle of my time ...,640,4.13,A+,"'Acceptance Rate: 7%, Net Price: $19998, SAT R...",0.126652,0.633135
1,#2 Best Colleges in America,Harvard University,Make sure to do your research before deciding ...,772,4.12,A+,"'Acceptance Rate: 5%, Net Price: $18037, SAT R...",0.208333,0.592187
2,#3 Best Colleges in America,Stanford University,"Now, 60+ years after a Stanford BSME, I am ast...","1,216",4.08,A+,"'Acceptance Rate: 4%, Net Price: $20023, SAT R...",0.412500,0.556250
3,#4 Best Colleges in America,Yale University,Yale is an amazing school! The professors and ...,"1,004",3.99,A+,"'Acceptance Rate: 6%, Net Price: $17511, SAT R...",0.322500,0.650833
4,#5 Best Colleges in America,Princeton University,"The courses, faculty, resources, and community...",445,3.96,A+,"'Acceptance Rate: 6%, Net Price: $18685, SAT R...",0.295833,0.522917


#### Return the total number of institutions with data webscrapped

In [14]:
sql = """
SELECT COUNT(*) AS 'Total number of institutions'
FROM niche_webscrapping
LIMIT 5
"""

pd.read_sql(sql, con = conn)

,Total number of institutions
0,820


#### Return the highest and lowest review ratings for these institutions

In [15]:
sql = """
SELECT instnm, 
    MAX(stu_ratings) AS 'Maximum student rating'
FROM niche_webscrapping
GROUP BY instnm
ORDER BY MAX(stu_ratings) DESC
LIMIT 5
"""

pd.read_sql(sql, con = conn)

,instnm,Maximum student rating
0,Everglades University,4.63
1,The Christ College of Nursing &amp; Health Sci...,4.34
2,Pomona College,4.22
3,Wabash College,4.15
4,Massachusetts Institute of Technology,4.13


In [16]:
sql = """
SELECT instnm, 
    MIN(stu_ratings) AS 'Minimum student rating'
FROM niche_webscrapping
GROUP BY instnm
ORDER BY MIN(stu_ratings) ASC
LIMIT 5
"""

pd.read_sql(sql, con = conn)

,instnm,Minimum student rating
0,Massachusetts Maritime Academy,2.98
1,California State University - Maritime Academy,3.05
2,University of Mary Washington,3.09
3,Maine Maritime Academy,3.15
4,Berea College,3.24


### Present the reviews with the highest and lowest polarity

In [17]:
sql = """
SELECT instnm, 
    MIN(review_polarity) AS 'Minimum student polarity'
FROM niche_webscrapping
GROUP BY instnm
ORDER BY MIN(review_polarity) ASC
LIMIT 5
"""

pd.read_sql(sql, con = conn)

,instnm,Minimum student polarity
0,University of Cincinnati,-0.440635
1,Berea College,-0.090046
2,Christian Brothers University,-0.089590
3,Buena Vista University,-0.030377
4,SUNY College of Environmental Science &amp; Fo...,-0.003287


In [18]:
sql = """
SELECT instnm, 
    MAX(review_polarity) AS 'Maximum student polarity'
FROM niche_webscrapping
GROUP BY instnm
ORDER BY MAX(review_polarity) DESC
LIMIT 5
"""

pd.read_sql(sql, con = conn)

,instnm,Maximum student polarity
0,Martin Luther College,0.779167
1,University of Evansville,0.741667
2,University of Wisconsin - Superior,0.712500
3,Franciscan Missionaries of Our Lady University,0.690000
4,Walsh University,0.687500


Prestigious institutions such as Massachussets Institute of Technology are among the institutions with the highest polarity and favorable reviews. Explore student reviews and the ranking and its underlying variables to identify which are related to better post graduation wages in the next step of the analysis (notebook 1b). 